3

In [1]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Missions à remplir
missions = [
    "Faire un post pour présenter @product",
    "Faire une story à propos de @product",
    "Prendre une photo de vous avec @product"
]

# Réponse à vérifier
response = "J'ai acheté @product hier et je suis très satisfait de mon achat. J'ai publié une photo avec @product sur Instagram et j'ai partagé une story à propos de mon expérience avec @product."

# Expression régulière pour extraire les tags et les mentions
pattern = r"(#\w+)|(@\w+)"

# Extraire les tags et les mentions de la réponse
response_tags_mentions = set(re.findall(pattern, response))

# Extraire le texte de la réponse
response_text = re.sub(pattern, "", response).strip()

# Vectoriser le texte des missions et de la réponse
vectorizer = TfidfVectorizer(stop_words='english')
mission_vectors = vectorizer.fit_transform(missions)
response_vector = vectorizer.transform([response_text])

# Entraîner un modèle de classification logistique pour prédire la mission appropriée
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
clf.fit(mission_vectors, np.arange(len(missions)))
prediction = clf.predict(response_vector)[0]

# Afficher la mission prédite
print(f"La réponse correspond à la mission : {missions[prediction]}")

La réponse correspond à la mission : Prendre une photo de vous avec @product


#  result

In [2]:

import joblib


import re
import numpy as np

import spacy
from torch import cosine_similarity
import torch
from Twitterpreprocessor import TwitterPreprocessor
nlp = spacy.load('en_core_web_sm')
from nltk.stem import PorterStemmer
ps = PorterStemmer()

missions = [
"Write a positive sentence about SaTT",
"Include #SaTT #crypto #influencer #Post2Earn #SocialFi ",
"Include “Earn crypto with your social networks” in your post "
]

response = " "
def porterstemmer(text):
  text = ' '.join(ps.stem(word) for word in text.split() if word in text)
  return text  

def lemmatization (text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)
def encode_text(text):
        # clean text from stop wods and punctuation
        text=TwitterPreprocessor(str(text)).fully_preprocess().text
        text=porterstemmer(text)
        text=lemmatization(text)
        text =[text]
        vectorizer = joblib.load('vectorizer.pkl')
        text = vectorizer.transform(text)
        text = torch.tensor(text.toarray())
        return text
def text_confirm(missions,response):
    # Expression régulière pour extraire les tags et les mentions
    pattern = r"([@#]\w+)"
    lien_pattern= re.compile(
            r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
            r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')
    # Extraire les tags et les mentions de la réponse
    response_tags_mentions = set(re.findall(pattern, response))
    response_lien = set(re.findall(lien_pattern, response))
    # Extraire le texte de la réponse
    response_text = re.sub(pattern, "", response).strip()
    response_text = re.sub(lien_pattern, "", response_text).strip()
    # Initialiser le score de validation à 0
    validation_score = 0
    total=0
    # Vérifier chaque mission
    for mission in missions:
        # Extraire les tags et les mentions de la mission
        mission_tags_mentions = set(re.findall(pattern, mission))
        total+=len(mission_tags_mentions)
    
        mission_lien = set(re.findall(lien_pattern, mission))
        total+=len(mission_lien)
    
        # Vérifier si tous les tags et mentions de la mission sont présents dans la réponse
        validation_score +=len(mission_tags_mentions.intersection(response_tags_mentions))
        validation_score +=len(mission_lien.intersection(response_lien))
        mission_text = re.sub(pattern,"", mission).strip()
        mission_text = re.sub(lien_pattern,"", mission).strip()
        mission_vector = encode_text(mission_text)
        response_vector = encode_text(response_text)
        # Print the shapes
        
        mission_vector = mission_vector.float()
        response_vector = response_vector.float()
  
        # Calcul de la similarité cosinus
        # similarity = cosine_similarity(mission_vector, response_vector)[0][0]
        similarity = np.dot(mission_vector, response_vector.T) / (np.linalg.norm(mission_vector) * np.linalg.norm( response_vector.T))
        total+=2
        if similarity > 0.7:
            # Ajouter un point si la similarité est suffisante
            validation_score += 1
    clf = joblib.load('postclf.pkl')
    predicted=clf.predict(response_vector)
    
    validation_score += 1 if predicted[0] == 1 else 0
    # Afficher le score de validation
    return round(validation_score *5 / total)
print(text_confirm(missions,response))

C:\Users\Chams\AppData\Local\Temp\ipykernel_20112\3725405068.py:78: RuntimeWarning: invalid value encountered in divide
  similarity = np.dot(mission_vector, response_vector.T) / (np.linalg.norm(mission_vector) * np.linalg.norm( response_vector.T))


0


# tester avec chatgbt

In [8]:
import os
import openai
import tweepy
from decouple import config

oracle=0
id_poste=0
def get_poste(oracle,id_poste):
    desc=""
    if oracle == 0:
        consumer_key = config('TWITTER_CONSUMER_KEY')
        consumer_secret = config('TWITTER_CONSUMER_SECRET')
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        # auth.set_access_token(access_key,access_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
    elif oracle == 1:
        oracle=0
    return desc   


    
def text_confirm():
  mission=[]
  poste=get_poste(oracle,id_poste)
  openai.api_key = config('OpenAI')
  start_sequence = "\nA:"
  restart_sequence = "\n\nQ: "
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Q:donnes un score sur 5  pour la confirmiter de ce poste a ces mission :\nposte: {poste} \n mission:{mission}\nA:",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
  )
  response = list(response)
  return response["choices"][0]["text"]

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " 4/5"
    }
  ],
  "created": 1684158984,
  "id": "cmpl-7GSyWKZ6gfiG4ysiZcFBwuiADKEgj",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 3,
    "prompt_tokens": 129,
    "total_tokens": 132
  }
}
